<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/%E4%B8%AD%E5%8F%A4%E3%83%9E%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%B3%E3%81%AE%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

準備

In [61]:
#各モジュールの取り込み
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
import re

In [4]:
#データの読み込み
paths = glob('/content/drive/MyDrive/中古マンション価格予想/data/input/train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv('/content/drive/MyDrive/中古マンション価格予想/data/input/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
#データ形状の確認
print(train_df.shape)
print(test_df.shape)

(1385764, 28)
(22395, 27)


In [9]:
#最大表示列数の設定
pd.set_option('display.max_columns', 200)

train_df.head(1)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,土地の形状,間口,延床面積（㎡）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,2012820,中古マンション等,NaN,2201,青森県,青森市,本町,青森,16,２ＤＫ,50,NaN,NaN,NaN,昭和61年,ＳＲＣ,住宅,NaN,NaN,NaN,NaN,商業地域,80.0,400.0,2010年第3四半期,改装済,NaN,6.875061


データ整理

In [16]:
# 各列の中身が何種類あるかを表示し、1種類しか持たないものを[]に表示
collist = []
for colname in train_df.columns:
  lencol = len(train_df[colname].unique())
  print(colname,lencol)
  if lencol == 1 and colname != '取引価格（総額）_log':
    collist.append(colname)

print(collist)

ID 692882
種類 1
地域 1
市区町村コード 619
都道府県名 47
市区町村名 619
地区名 12136
最寄駅：名称 3859
最寄駅：距離（分） 46
間取り 64
面積（㎡） 208
土地の形状 1
間口 1
延床面積（㎡） 1
建築年 78
建物の構造 13
用途 23
今後の利用目的 5
前面道路：方位 1
前面道路：種類 1
前面道路：幅員（ｍ） 1
都市計画 17
建ぺい率（％） 7
容積率（％） 18
取引時点 62
改装 3
取引の事情等 10
取引価格（総額）_log 382
['種類', '地域', '土地の形状', '間口', '延床面積（㎡）', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）']


In [ ]:
#上記の特徴量を排除
train_df = train_df.drop(['種類', '地域', '土地の形状', '間口', '延床面積（㎡）', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）'],axis = 1)
test_df = test_df.drop(['種類', '地域', '土地の形状', '間口', '延床面積（㎡）', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）'],axis = 1)

In [56]:
#和暦変換の無名関数
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

In [57]:
#面積を変換する無名関数
def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

In [58]:
#最寄り駅からの距離を変換する無名関数
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(float(moyori))
    return moyori

In [63]:
train_df['取引時点_何年前'] = train_df['取引時点'].apply(lambda x: 2021-int(x[:4]))
train_df.drop(['取引時点'], axis=1, inplace=True)
train_df['建築年'] = train_df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
train_df['面積（㎡）'] = train_df['面積（㎡）'].apply(lambda x: normalize_area(x))
train_df['最寄駅：距離（分）'] = train_df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

In [55]:
train_df.head(1)

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,改装,取引の事情等,取引価格（総額）_log,取引時点_何年前
0,2012820,2201,青森県,青森市,本町,青森,16,２ＤＫ,50,1986.0,ＳＲＣ,住宅,NaN,商業地域,80.0,400.0,改装済,NaN,6.875061,11


In [64]:
#特徴量がobject形式のものを抽出
object_df=train_df.select_dtypes(include='object')
object_df.head(1)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,青森県,青森市,本町,青森,２ＤＫ,ＳＲＣ,住宅,NaN,商業地域,改装済,NaN
